<a href="https://colab.research.google.com/github/TheKVT/Archive-Extract/blob/main/Archive_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **<font color='blue'> Archive-Extract by [TheKVT](https://github.com/TheKVT/Archive-Extract) </font>**

## ![alt text](https://drive.google.com/favicon.ico?v=3) __Mount GDrive__

In [ ]:
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# #Extract Files

In [ ]:
# ============================= FORM ===========================
MODE = "UNZIP"  # @param ["UNZIP", "UNTAR", "UNRAR", "7Z", "ISO", "OTHER"]
PATH_TO_FILE = ""  # @param {type:"string"}
extractPath = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request

def checkAvailable(path_="", userPath=False):
    from os import path as _p

    if path_ == "":
        return False
    else:
        return (
            _p.exists(path_)
            if not userPath
            else _p.exists(f"/usr/local/sessionSettings/{path_}")
        )


def runSh(args, *, output=False, shell=False, cd=None):
    import subprocess, shlex

    if not shell:
        if output:
            proc = subprocess.Popen(
                shlex.split(args), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=cd
            )
            while True:
                output = proc.stdout.readline()
                if output == b"" and proc.poll() is not None:
                    return
                if output:
                    print(output.decode("utf-8").strip())
        return subprocess.run(shlex.split(args), cwd=cd).returncode
    else:
        if output:
            return (
                subprocess.run(
                    args,
                    shell=True,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.STDOUT,
                    cwd=cd,
                )
                .stdout.decode("utf-8")
                .strip()
            )
        return subprocess.run(args, shell=True, cwd=cd).returncode

def extractFiles():
    global extractPath
    if ARCHIVE_PASSWORD:
      passADD = f'-P {ARCHIVE_PASSWORD}'
    else:
      passADD = ''
    if not extractPath:
      extractPath = "/content/extract"
    os.makedirs(extractPath, exist_ok=True)
    if MODE == "UNZIP":
        runSh('unzip '+passADD+f' "{PATH_TO_FILE}" -d "{extractPath}"', output=True)
    elif MODE == "UNRAR":
        runSh(f'unrar x "{PATH_TO_FILE}" "{extractPath}" '+passADD+' -o+', output=True)
    elif MODE == "UNTAR":
        runSh(f'tar -C "{extractPath}" -xvf "{PATH_TO_FILE}"', output=True)
    else:
        runSh(f'7z x "{PATH_TO_FILE}" -o{extractPath} '+passADD, output=True)


extractFiles()

Some rare times this extractor shows error...
Then use following command <br>
1. ``` %cd your dictionary/folder of archived file ```
2. for any type of archived files this command is same <br>
```!7z x archived_file_name``` <br>
if it have password use this <br>
```!7z x archived_file_name -pfilePasswordHere``` <br>
3. In case of error only use this ⚠️

# #Archive Files

In [ ]:
# ============================= FORM ============================= #

MODE = "ZIP" #@param ["ZIP", "TAR", "7Z"]
FILENAME = ""  # @param {type:"string"}
PATH_TO_FILE = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# option supports b k m g (bytes, kilobytes, megabytes, gigabytes)
SPLIT = "no" #@param ["1g", "2g", "3g", "4g", "5g", "no"]


compress = 9#@param  {type:"slider", min:0, max:9, step:0}
#@markdown <font size=3%>Add multi files by use this to path separate between to this character `|`. Ex ```path/to/1 | path/to/2``` </br>
# ================================================================ #
from pathlib import PurePosixPath

pathList = PATH_TO_FILE.split('|')
if MODE == "ZIP":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE.ZIP"
    if ARCHIVE_PASSWORD:
      passADD = f'--password "{ARCHIVE_PASSWORD}"'
    else:
      passADD = ''
    splitC = f"-s {SPLIT}" if not 'no' in SPLIT else "" 
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && zip {passADD} -{compress} {splitC} -v -r -u "{FILENAME}" "{partName}"'
      !$cmd
elif MODE == "TAR":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE"
    cmd = f'GZIP=-{compress} tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}'
    !$cmd
else:
    if not FILENAME:
        FILENAME = "/content/NEW_FILE"
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && 7z a -mx={compress} "{FILENAME}.7z" "{partName}"'
      !$cmd

# <b>Get Archived File info </b>
<p>Know file names and sizes of any archived files even if it is protected</p>

In [ ]:
file_name = "" #@param {type:"string"}
Mode = "ZIP" #@param ["ZIP", "TAR", "RAR", "7Z"]

from zipfile import ZipFile
import datetime,humanize
import tarfile
import sys

if not file_name:
    print('Enter File Path First')
else:
      if Mode == "ZIP": 
        with ZipFile(file_name, 'r') as zip:
          for info in zip.infolist():
            print(info.filename)
            print('\tLast Modified:\t' + str(datetime.datetime(*info.date_time)))
            print('\tZIP version:\t' + str(info.create_version))
            print('\tSystem:\t\t' + str(info.create_system) + '  (0 = Windows, 3 = Unix)')
            print('\tCompressed:\t' + humanize.naturalsize(str(info.compress_size)))
            print('\tUncompressed:\t' + humanize.naturalsize(str(info.file_size)))
            print(u'\u2500'* 60)
      elif Mode == "TAR":  
             t = tarfile.open('/content/sample.tar', 'r')
             for info in t.getmembers():
              print(u'\u2500'* 60)
              print(info.name)
              print('Modified:', time.ctime(info.mtime))
              print('Mode    :', oct(info.mode))
              print('Type    :', info.type)
              print('Size    :', humanize.naturalsize(info.size)) 
      else:
        !7z l "{filepath}"